In [1]:
import tensorflow as tf
import numpy as np
import os
import utils
import gpfunctions as gp

In [2]:
def gen_data(n_data, dim, n_gp_samples, n_mc_samples, l, kernel):
    X = np.random.uniform(low = -1.0, high = 1.0, size = (n_data, n_gp_samples, dim))
    Y = np.random.uniform(low = -1.0, high = 1.0, size = (n_data, n_gp_samples))

    K_XX = kernel("np",X,X,l)
    A = np.linalg.solve(K_XX, np.expand_dims(Y,axis=2))
    
    mc_samples = np.random.uniform(low = -1.0, high = 1.0, size = [1, n_mc_samples, dim])
    y = gp.GP("np",X,A,mc_samples,l, kernel)

    min_vals = np.min(y, axis = 1).reshape(n_data,1)
    max_vals = np.max(y, axis = 1).reshape(n_data,1)

    return (X, A, min_vals, max_vals)

In [5]:
data_dir = utils.loadConfig()['DATA_DIR']
def run_gen_data_for(dim, no_training, no_testing, n_gp_samples, n_mc_samples, length, kernelstr, kernel):
    no_data = [no_training,no_testing]
    for i in range(2):
        X, A, minv, maxv = gen_data( no_data[i], dim, n_gp_samples, n_mc_samples, length, kernel)
        s = 'training' if i == 0 else 'testing'
        directory = utils.getDataPath(dim, s, kernelstr)
        if not os.path.exists(directory):
            os.makedirs(directory)

        np.save( directory + "/X", X)
        np.save( directory + "/A", A)
        np.save( directory + "/minv", minv)
        np.save( directory + "/maxv", maxv)

In [13]:
for dim in [1,2,3,4,6]:
    for kernel_id in [0,1,2]:
        #dim = 2
        no_training = 2000
        no_testing = 2000
        n_bumps = 6
        n_mc_samples = 1000
        length = 2/n_bumps*np.sqrt(dim)

        #kernel_id = 2
        kernelstr = ["rbf", "matern32", "matern52"][kernel_id]
        kernel = [gp.rbf_kernel, gp.matern32_kernel, gp.matern52_kernel][kernel_id]

        run_gen_data_for(dim, no_training, no_testing, n_bumps, n_mc_samples, length, kernelstr, kernel)
        print(dim)

1
1
1
2
2
2
3
3
3
4
4
4
6
6
6
